In [2]:
# -*- coding: utf-8 -*-
"""
Energy Management System Scoring Tool (Enhanced with Energy Cost Penalty and Violation Handling by Timestamp)

This script evaluates energy management strategies for microgrid systems by calculating
how well a strategy balances power sources (solar, diesel, grid) while maintaining
battery state of charge within safe limits. Now includes:
    - Energy Cost penalty
    - If SOC < DOD, only that timestep is excluded from score calculation (not whole site)
    - Reports which timesteps were excluded for each site
"""

import csv
import sys
import os
import logging
import argparse
from config import *

# ---------------- Logging Setup ----------------

def setup_logging(log_file=DEFAULT_LOG_FILE, log_level=DEFAULT_LOG_LEVEL, console=LOG_TO_CONSOLE):
    numeric_level = getattr(logging, log_level.upper(), None)
    if not isinstance(numeric_level, int):
        raise ValueError(f'Invalid log level: {log_level}')
    logger = logging.getLogger()
    logger.setLevel(numeric_level)
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
    formatter = logging.Formatter(LOG_FORMAT, LOG_DATE_FORMAT)
    if console:
        console_handler = logging.StreamHandler()
        console_handler.setLevel(numeric_level)
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)
    if log_file:
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(numeric_level)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
    return logger

# ---------------- Data Readers ----------------

def read_real_power(load_path):
    load_dict = {}
    logger = logging.getLogger()
    logger.info(f"Reading power consumption data from {load_path}")
    try:
        with open(load_path) as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader)
            row_count = 0
            for row in csv_reader:
                key1, key2, value = row[0], row[1], row[2]
                if key1 not in load_dict:
                    load_dict[key1] = {}
                    logger.debug(f"Added new site to power data: {key1}")
                load_dict[key1][key2] = float(value) * POWER_CONVERSION_FACTOR
                row_count += 1
            logger.info(f"Successfully read {row_count} power consumption records")
        return load_dict
    except UnicodeDecodeError:
        logger.critical("Failed to read power data: Unknown file type or encoding issue")
        sys.exit(1)
    except FileNotFoundError:
        logger.critical(f"Power consumption data file not found: {load_path}")
        sys.exit(1)
    except Exception as e:
        logger.critical(f"Unexpected error reading power data: {str(e)}")
        sys.exit(1)

def read_reference_data(reference_path):
    load_dict = {}
    solar_dict = {}
    site_config = {}
    count = 0
    logger = logging.getLogger()
    logger.info(f"Reading reference data from {reference_path}")
    try:
        with open(reference_path) as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader)
            logger.debug(f"Reference data header: {header}")
            in_config_section = False
            for row in csv_reader:
                if not row or all(cell.strip() == "" for cell in row):
                    logger.debug("Skipping empty row")
                    continue
                row_str = ','.join(row).lower()
                if any(keyword in row_str for keyword in ['batterycapacity', 'dieselpower', 'gridpower', 'initsoc', 'dod']):
                    logger.debug(f"Detected site configuration header row: {row}")
                    in_config_section = True
                    continue
                if not in_config_section:
                    count += 1
                    key1, key2, value, value1 = row[0], row[1], row[2], row[3]
                    if key1 not in load_dict:
                        load_dict[key1] = {}
                        solar_dict[key1] = {}
                        logger.debug(f"Added new site to reference data: {key1}")
                    load_dict[key1][key2] = float(value) * POWER_CONVERSION_FACTOR
                    solar_dict[key1][key2] = float(value1) * POWER_CONVERSION_FACTOR
                    continue
                site_name = row[0]
                logger.info(f"Processing configuration for site: {site_name}")
                try:
                    diesel_power = float(row[1]) * POWER_CONVERSION_FACTOR
                    rated_capacity = float(row[2])
                    rated_voltage = float(row[3])
                    coe = float(row[4])
                    dis_coe = float(row[5])
                    init_soc = float(row[6])
                    dod = float(row[7])
                    grid_power = float(row[8]) * POWER_CONVERSION_FACTOR
                    grid_plan = row[9][1:len(row[9]) - 1].split(" ")
                    plan = [grid_plan[i].lower() == "true" for i in range(len(grid_plan))]
                    config = {
                        "site": site_name,
                        "dieselPower": diesel_power,
                        "initSoc": init_soc,
                        "ratedCapacity": rated_capacity,
                        "ratedVoltage": rated_voltage,
                        "DOD": dod,
                        "coe": coe,
                        "disCoe": dis_coe,
                        "gridPower": grid_power,
                        "gridPlan": plan,
                        "solarPower": solar_dict[site_name],
                        "loadPower": load_dict[site_name]
                    }
                    site_config[site_name] = config
                except (ValueError, IndexError) as e:
                    logger.error(f"Invalid configuration data for site {site_name}: {str(e)}")
                    logger.warning(f"Skipping site {site_name} due to invalid configuration")
                    continue
            logger.info(f"Successfully processed {len(site_config)} site configurations")
        return site_config
    except UnicodeDecodeError:
        logger.critical("Failed to read reference data: Unknown file type or encoding issue")
        sys.exit(1)
    except FileNotFoundError:
        logger.critical(f"Reference data file not found: {reference_path}")
        sys.exit(1)
    except Exception as e:
        logger.critical(f"Unexpected error reading reference data: {str(e)}")
        sys.exit(1)

# ---------------- Strategy Reader ----------------

def read_strategy_data(strategy_path):
    strategy_dict = {}
    logger = logging.getLogger()
    logger.info(f"Reading strategy data from {strategy_path}")
    try:
        with open(strategy_path) as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader)
            logger.debug(f"Strategy data header: {header}")
            row_count = 0
            site_count = 0
            prev_site = None
            for row in csv_reader:
                site_name = row[0]
                index = row[1]
                grid = row[2].lower() == "true"
                diesel = row[3].lower() == "true"
                solar = row[4].lower() == "true"
                battery_status = {
                    "useGrid": grid,
                    "useDiesel": diesel,
                    "useSolar": solar
                }
                if site_name not in strategy_dict:
                    strategy_dict[site_name] = {}
                    site_count += 1
                    logger.debug(f"Added new site to strategy data: {site_name}")
                strategy_dict[site_name][index] = battery_status
                row_count += 1
                if prev_site != site_name:
                    prev_site = site_name
            logger.info(f"Successfully read {row_count} strategy records for {site_count} sites")
        return strategy_dict
    except UnicodeDecodeError:
        logger.critical("Failed to read strategy data: Unknown file type or encoding issue")
        sys.exit(1)
    except FileNotFoundError:
        logger.critical(f"Strategy data file not found: {strategy_path}")
        sys.exit(1)
    except Exception as e:
        logger.critical(f"Unexpected error reading strategy data: {str(e)}")
        sys.exit(1)

# ---------------- Scoring Helpers ----------------

def get_diesel_num(strategy, predict_length):
    diesel_num = 0.0
    if strategy['0']["useDiesel"]:
        diesel_num += 1
    for i in range(0, predict_length * QUARTERS_PER_HOUR - 1):
        battery_status = strategy[str(i)]
        next_battery_status = strategy[str(i + 1)]
        if (not battery_status["useDiesel"]) and next_battery_status["useDiesel"]:
            diesel_num += 1
    return diesel_num

def get_total_diesel_time(strategy, predict_length):
    total_time = 0.0
    for i in range(0, predict_length * QUARTERS_PER_HOUR):
        battery_status = strategy[str(i)]
        if battery_status["useDiesel"]:
            total_time += MINUTES_PER_QUARTER
    return total_time

def get_max_time(strategy, predict_length):
    max_time = 0.0
    i = 0
    while i < predict_length * QUARTERS_PER_HOUR:
        battery_status = strategy[str(i)]
        if not battery_status["useDiesel"]:
            i += 1
            continue
        cur_time = 0.0
        while i < predict_length * QUARTERS_PER_HOUR and strategy[str(i)]["useDiesel"]:
            cur_time += MINUTES_PER_QUARTER
            i += 1
        max_time = max(max_time, cur_time)
    return max_time

def get_total_grid_time(strategy, predict_length):
    total_time = 0.0
    for i in range(0, predict_length * QUARTERS_PER_HOUR):
        battery_status = strategy[str(i)]
        if battery_status["useGrid"]:
            total_time += MINUTES_PER_QUARTER
    return total_time

# ---------------- Score Calculation ----------------

def calc_single_site_score(strategy, config, predict_length):
    current_soc = float(config["initSoc"])
    capacity = float(config["ratedCapacity"]) * float(config["ratedVoltage"])
    site_name = config["site"]
    total_energy_cost = 0.0
    violations = []  # track SOC violations
    valid_timesteps = 0  # count only valid timesteps
    for i in range(0, predict_length * QUARTERS_PER_HOUR):
        coe = float(config["coe"])
        battery_status = strategy[str(i)]
        hour_of_simulation = (i // QUARTERS_PER_HOUR) % len(config["solarPower"])
        diesel_power = config["dieselPower"] if battery_status["useDiesel"] else 0.0
        solar_power = (config["solarPower"][str(hour_of_simulation)] if battery_status["useSolar"] else 0.0)
        hour_of_day = (i // QUARTERS_PER_HOUR) % len(config["gridPlan"])
        grid_power = (config["gridPower"] if battery_status["useGrid"] and config["gridPlan"][hour_of_day] else 0.0)
        load_power = config["loadPower"][str(hour_of_simulation)]
        net_power = diesel_power + solar_power + grid_power - load_power
        if net_power < 0:
            coe = float(config["disCoe"])
        current_soc += net_power * coe / (QUARTERS_PER_HOUR * capacity)
        current_soc = min(current_soc, MAX_SOC)
        if current_soc < config["DOD"]:
            violations.append(i)  # record violation timestep
            continue  # skip this timestep in score calc
        # --- Energy cost calculation per 15-min step ---
        energy_cost_t = 0.0
        if battery_status["useGrid"] and config["gridPlan"][hour_of_day]:
            energy_cost_t += (config["gridPower"] / 1000.0) * 0.25 * GRID_UNIT_COST
        if battery_status["useDiesel"]:
            energy_cost_t += (config["dieselPower"] / 1000.0) * 0.25 * DIESEL_UNIT_COST
        if battery_status["useSolar"]:
            energy_cost_t += (config["solarPower"][str(hour_of_simulation)] / 1000.0) * 0.25 * SOLAR_UNIT_COST
        total_energy_cost += energy_cost_t
        valid_timesteps += 1
    diesel_num = get_diesel_num(strategy, predict_length)
    diesel_time = get_total_diesel_time(strategy, predict_length)
    max_diesel_time = get_max_time(strategy, predict_length)
    grid_time = get_total_grid_time(strategy, predict_length)
    score = (DIESEL_START_PENALTY * diesel_num +
             DIESEL_RUNTIME_PENALTY * diesel_time +
             MAX_DIESEL_RUN_PENALTY * max_diesel_time +
             GRID_USAGE_PENALTY * grid_time +
             W5_ENERGY_COST * total_energy_cost)
    return score, violations, valid_timesteps

# ---------------- Overall Calculator ----------------

def score_calculator(strategy_path, reference_path, days, num_sites=DEFAULT_NUM_SITES):
    total_time = DAYS_PER_WEEK * HOURS_PER_DAY * QUARTERS_PER_HOUR
    predict_length = HOURS_PER_DAY * days
    site_config = read_reference_data(reference_path)
    site_strategy = read_strategy_data(strategy_path)
    if not site_strategy:
        return -1.0, {}
    actual_sites = len(site_strategy.keys())
    if num_sites != actual_sites:
        num_sites = actual_sites
    result = 0.0
    site_results = {}
    for site_name in site_strategy.keys():
        if site_name not in site_config:
            site_results[site_name] = {"score": None, "violations": ["Missing config"]}
            continue
        if len(site_strategy[site_name]) != total_time:
            site_results[site_name] = {"score": None, "violations": ["Strategy length mismatch"]}
            continue
        site_score, violations, valid_steps = calc_single_site_score(site_strategy[site_name], site_config[site_name], predict_length)
        site_results[site_name] = {"score": site_score, "violations": violations, "valid_steps": valid_steps}
        result += site_score
    final_score = result / num_sites
    return final_score, site_results

# ---------------- CLI ----------------

# Remove the parse_arguments function as we will not be using command-line arguments

def main():
    # Directly define file paths and parameters for use in the notebook
    reference_path = '/content/reference_data.csv'
    strategy_path = '/content/strategy_submission.csv'
    num_sites = DEFAULT_NUM_SITES  # Use the default or define a specific number
    days = DEFAULT_EVALUATION_DAYS  # Use the default or define a specific number
    log_file = DEFAULT_LOG_FILE # Use the default or specify a log file
    log_level = DEFAULT_LOG_LEVEL # Use the default or specify a log level
    console_logging = LOG_TO_CONSOLE # Use the default or specify console logging

    logger = setup_logging(log_file=log_file, log_level=log_level, console=console_logging)
    logger.info("Energy Management System Scoring Tool starting")
    final_score, site_results = score_calculator(strategy_path, reference_path, days, num_sites)
    print("\n--- Evaluation Results ---")
    print(f"Final Average Score: {final_score}")
    for site, res in site_results.items():
        print(f"\nSite: {site}")
        print(f"  Score: {res['score']}")
        print(f"  Valid Timesteps: {res.get('valid_steps','N/A')}")
        if res["violations"]:
            print(f"  Violations (SOC<DOD) at timesteps: {res['violations']}")
        else:
            print("  No violations")
    logger.info("Evaluation complete")

if __name__ == "__main__":
    main()

2025-10-31 18:30:38 - root - INFO - ipython-input-3960157989.py:main:327 - Energy Management System Scoring Tool starting
2025-10-31 18:30:38 - root - INFO - ipython-input-3960157989.py:read_reference_data:79 - Reading reference data from /content/reference_data.csv
2025-10-31 18:30:38 - root - INFO - ipython-input-3960157989.py:read_reference_data:106 - Processing configuration for site: site1
2025-10-31 18:30:38 - root - INFO - ipython-input-3960157989.py:read_reference_data:106 - Processing configuration for site: site2
2025-10-31 18:30:38 - root - INFO - ipython-input-3960157989.py:read_reference_data:106 - Processing configuration for site: site3
2025-10-31 18:30:38 - root - INFO - ipython-input-3960157989.py:read_reference_data:106 - Processing configuration for site: site4
2025-10-31 18:30:38 - root - INFO - ipython-input-3960157989.py:read_reference_data:106 - Processing configuration for site: site5
2025-10-31 18:30:38 - root - INFO - ipython-input-3960157989.py:read_reference


--- Evaluation Results ---
Final Average Score: 45856.65

Site: site1
  Score: 51768.0
  Valid Timesteps: 672
  No violations

Site: site10
  Score: 43182.0
  Valid Timesteps: 672
  No violations

Site: site2
  Score: 56439.0
  Valid Timesteps: 672
  No violations

Site: site3
  Score: 40617.0
  Valid Timesteps: 672
  No violations

Site: site4
  Score: 47973.0
  Valid Timesteps: 665
  Violations (SOC<DOD) at timesteps: [253, 254, 255, 256, 257, 258, 259]

Site: site5
  Score: 52707.0
  Valid Timesteps: 672
  No violations

Site: site6
  Score: 49434.0
  Valid Timesteps: 672
  No violations

Site: site7
  Score: 50691.0
  Valid Timesteps: 672
  No violations

Site: site8
  Score: 26680.5
  Valid Timesteps: 672
  No violations

Site: site9
  Score: 39075.0
  Valid Timesteps: 672
  No violations
